In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [9]:
## load the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [11]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
## Encode Categorical Variables
lable_encoder = LabelEncoder()
df['Gender'] = lable_encoder.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [13]:
## if we use lableencoding in case of Geography column there will be issue as numerical values will be considered
## as priority order which will create a issue here.

In [14]:
## one hot encoder
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoded = one_hot_encoder_geo.fit_transform(df[['Geography']])
geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [15]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [16]:
geo_df = pd.DataFrame(geo_encoded.toarray(),columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

In [17]:
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [18]:
## Combine all the one hot enocoded columns with the original data

In [19]:
df = pd.concat([df.drop('Geography',axis=1),geo_df],axis=1)

In [20]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [21]:
## save the encoders and scaler
with open('lable_encoder_gender.pkl','wb') as file:
    pickle.dump(lable_encoder,file)
with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

In [22]:
## divide the dataset into dependent and indepedent features

In [43]:
x = df.drop('Exited', axis=1).values.astype('float32') 
y = df['Exited']

In [102]:
## split the data into training and testing sets
from sklearn.preprocessing import StandardScaler
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)


## scale these features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [103]:
type(x_test)

numpy.ndarray

In [104]:
x_test

array([[-0.5774961 ,  0.9132475 , -0.65578586, ..., -0.9985011 ,
         1.7257231 , -0.576388  ],
       [-0.29729736,  0.9132475 ,  0.39001092, ...,  1.0015011 ,
        -0.5794672 , -0.576388  ],
       [-0.5256074 , -1.0949934 ,  0.48508334, ..., -0.9985011 ,
        -0.5794672 ,  1.7349424 ],
       ...,
       [ 0.8131198 , -1.0949934 ,  0.7703006 , ...,  1.0015011 ,
        -0.5794672 , -0.576388  ],
       [ 0.41876608,  0.9132475 , -0.9410032 , ...,  1.0015011 ,
        -0.5794672 , -0.576388  ],
       [-0.2454087 ,  0.9132475 ,  0.00972116, ..., -0.9985011 ,
         1.7257231 , -0.576388  ]], dtype=float32)

In [105]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implementation

In [106]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [107]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [108]:
## build our ANN Model
model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ## HL1 Connected to input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid') ## Output layer
])

In [109]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [110]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [111]:
## compile  the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [124]:
## set up the tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [125]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [126]:
## set up Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [127]:
print("x_train:", type(x_train))
print("y_train:", type(y_train))
print("x_test:", type(x_test))
print("y_test:", type(y_test))

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

x_train: <class 'numpy.ndarray'>
y_train: <class 'numpy.ndarray'>
x_test: <class 'numpy.ndarray'>
y_test: <class 'numpy.ndarray'>


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [128]:
## Train the model
history = model.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 767us/step - loss: 0.3392 - accuracy: 0.8594 - val_loss: 0.3363 - val_accuracy: 0.8600
Epoch 2/100
250/250 [==============================] - 0s 658us/step - loss: 0.3370 - accuracy: 0.8608 - val_loss: 0.3388 - val_accuracy: 0.8650
Epoch 3/100
250/250 [==============================] - 0s 704us/step - loss: 0.3316 - accuracy: 0.8633 - val_loss: 0.3602 - val_accuracy: 0.8615
Epoch 4/100
250/250 [==============================] - 0s 663us/step - loss: 0.3307 - accuracy: 0.8605 - val_loss: 0.3557 - val_accuracy: 0.8530
Epoch 5/100
250/250 [==============================] - 0s 654us/step - loss: 0.3297 - accuracy: 0.8669 - val_loss: 0.3384 - val_accuracy: 0.8640
Epoch 6/100
250/250 [==============================] - 0s 703us/step - loss: 0.3251 - accuracy: 0.8676 - val_loss: 0.3502 - val_accuracy: 0.8610


In [129]:
model.save('model.h5')

/Users/vaibhavkale/Desktop/DL_Projects/ANN_Classification/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [130]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [131]:
%tensorboard --logdir logs/fit 

Reusing TensorBoard on port 6006 (pid 29056), started 0:05:16 ago. (Use '!kill 29056' to kill it.)